# BERT for Question Answer

## Setup

In [1]:
# if using colab, uncomment the below
# !pip install torch "argilla" datasets accelerate transformers setfit
# !pip install wandb

In [2]:
from datasets import load_dataset

In [3]:
squadv2 = load_dataset('squad_v2')

Found cached dataset squad_v2 (/Users/arths/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(squadv2)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

## Preprocessing Data

Our sequences will look like

```
[CLS] ...question tokens... [SEP] ...context tokens... [SEP]
```

In cases where the context is too long, we'll split into multiple sequences, like

```
[CLS] ...question tokens... [SEP] ...some context tokens... [SEP]
[CLS] ...question tokens... [SEP] ...overlap from prev sequence... ...more context tokens... [SEP]
...
```

Bassed on the question tokens, the model needs to get a contiguous subset of the context tokens as the answer. Our dataset contains the start position of the answer in the original context string.

The HuggingFace tokenizer is able to map each item in the tokenized sequence to the start and end indices in the original context string.

We need to find which indices in the tokenized sequence map to the start and end of the answer so that our model knows how to predict the contiguous answer section.

If there is no answer available in a sequence, we will set the answer start and end to the `[CLS]` token.

Additionally, for context split accross multiple tokenized sequences, for sequences without the answer (or with only a part of the answer), we will treat it the same as 'no answer' sequences.

In [6]:
def map_answer(offset, ans_start, ans_end, sequence_ids):

    # get start and end indices in tokenized sequence
    idx = 0
    while sequence_ids[idx] != 1: idx += 1
    context_start = idx
    while sequence_ids[idx] == 1: idx += 1
    context_end = idx - 1

    # start with [CLS]
    start, end = 0, 0

    # if answer is not fully in this tokenized sequence, map to [CLS]
    if offset[context_end][0] > ans_end or offset[context_end][1] < ans_start:
        return start, end
    
    idx = context_start
    while idx <= context_end and offset[idx][0] <= ans_start: idx += 1
    start = idx - 1

    idx = context_end
    while idx >= context_start and offset[idx][1] >= ans_end: idx -= 1
    end = idx + 1

    return start, end

def get_answer_mapped_data(batch):
    questions = batch['question']
    contexts = batch['context']
    answers = batch['answers']

    inputs = tokenizer(
        # add data for tokenizing and padding
        questions, contexts,        # data to tokenize
        max_length=400,             # max_length per sequence
        padding='max_length',       # pad til max_length

        # handling truncation
        truncation='only_second',   # only truncate context
        stride=128,                 # overlap size
        return_overflowing_tokens=True, # tokenizer automatically 
                                        # makes extra sequences

        # get mappings to original sentence
        return_offsets_mapping=True,# used to map answer to sequence
    )

    offset_mapping = inputs.pop('offset_mapping')
    sample_map = inputs.pop('overflow_to_sample_mapping')
    starts = []
    ends = []

    for i, offset in enumerate(offset_mapping):

        map_i = sample_map[i]

        answer = answers[map_i]
        text = answer['text']
        
        # SQuAD v2 has some adversarial examples with 'unanswerable' questions
        # in this case, map to [CLS]
        if len(text) < 1:
            starts.append(0)
            ends.append(0)
            continue

        ans_start = answer['answer_start'][0]
        ans_end = ans_start + len(text[0])
        sequence_ids = inputs.sequence_ids(map_i)

        start, end = map_answer(offset, ans_start, ans_end, sequence_ids)

        starts.append(start)
        ends.append(end)

    inputs['start_positions'] = starts
    inputs['end_positions'] = ends

    return inputs

In [7]:
tokenized_squadv2 = squadv2.map(get_answer_mapped_data,
                                batched=True,
                                remove_columns=squadv2['train'].column_names)

Loading cached processed dataset at /Users/arths/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-9b81bd08df7dc75b.arrow
Loading cached processed dataset at /Users/arths/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-7f101e47c6d41ad5.arrow


In [8]:
print(tokenized_squadv2['train'][0].keys())

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])


# Train

### Set Up HuggingFace Training

In [9]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import DefaultDataCollator

We will use DistilBERT for lower memory usage and thus faster training (from larger batch sizes).

In [10]:
dbert_qa = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [11]:
BATCH_SIZE = 16
LR = 2e-5
EPOCHS = 3
WEIGHT_DECAY = 0.01
CHKPT_DIR = 'checkpoints'

In [12]:
data_collator = DefaultDataCollator()

train_args = TrainingArguments(
    # save model every epoch
    output_dir=CHKPT_DIR,
    save_strategy='epoch',

    # epochs
    evaluation_strategy='epoch',
    num_train_epochs=EPOCHS,

    # batch sizes
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    
    # hyperparams
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY,

    # log to wandb
    report_to='wandb',
)

trainer = Trainer(
    model=dbert_qa,
    args=train_args,
    train_dataset=tokenized_squadv2['train'],
    eval_dataset=tokenized_squadv2['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

### Run Training

In [13]:
import wandb

# use to log in to wandb if needed
# API_KEY = # wandb api key
# wandb.login(key=API_KEY)

wandb.init(
    project='SQuAD2.0 with Fine-Tuned DistilBERT',
    notes='Solving Standford\'s SQuAD 2.0 Q&A dataset with DistilBERT transfer learning.',
)

wandb.config = {
    'epochs': EPOCHS, 
    'learning_rate': LR, 
    'batch_size': BATCH_SIZE,
    'weight_decay': WEIGHT_DECAY,
}

trainer.train()

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arth-shukla. Use `wandb login --relogin` to force relogin


c:\Users\Arth Shukla\anaconda3\envs\bert-transfer-learning\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 500/24654 [02:13<1:43:07,  3.90it/s]

{'loss': 0.3954, 'learning_rate': 1.9594386306481706e-05, 'epoch': 0.06}


  4%|▍         | 1000/24654 [04:21<1:41:50,  3.87it/s]

{'loss': 0.2009, 'learning_rate': 1.9188772612963417e-05, 'epoch': 0.12}


...

                                                       
100%|██████████| 24654/24654 [1:49:01<00:00,  4.29it/s]

{'eval_loss': 0.14294327795505524, 'eval_runtime': 62.9034, 'eval_samples_per_second': 192.422, 'eval_steps_per_second': 12.034, 'epoch': 3.0}


100%|██████████| 24654/24654 [1:49:02<00:00,  3.77it/s]

{'train_runtime': 6542.3196, 'train_samples_per_second': 60.291, 'train_steps_per_second': 3.768, 'train_loss': 0.13702084790512412, 'epoch': 3.0}


eval/loss,▁▃█
eval/runtime,█▃▁
eval/samples_per_second,▁▆█
eval/steps_per_second,▁▆█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


## Inference

In a previous project, I fine-tuned DistilBERT for Sentiment Analysis on Sentiment140 (https://github.com/arth-shukla/sentiment140-bert-transfer-learning). Here, I found that only 1-2 epochs were needed for optimal fine-tuning.

Similarly, as seen above, after the first epoch validation loss only increases. It's likely the model starts overfitting here. So, I load the model from the end of the first epoch.

Feel free to play around with the model below! It seems to have the most trouble with 
1. questions using language it hasn't seen before (e.g. non-American names)
2. deducing "unanswerable" questions.
3. switching around context in a sentence

In [13]:
# TODO: add code to install my model checkpoints from gdrive

In [15]:
from transformers import pipeline

trained_dbert_qa = AutoModelForQuestionAnswering.from_pretrained('./checkpoints/checkpoint-8218')
question_answerer = pipeline('question-answering', model=trained_dbert_qa, tokenizer=tokenizer)

In [16]:
# Information from https://en.wikipedia.org/wiki/List_of_highest-grossing_films
c1 = 'With a worldwide box-office gross of over $2.9 billion, Avatar is proclaimed to be the "highest-grossing" film, but such claims usually refer to theatrical revenues only and do not take into account home video and television income, which can form a significant portion of a film\'s earnings.'

q1a = 'How much did Avatar gross worldwide?'
q1a_ans = question_answerer(question=q1a, context=c1)
print('q1a', q1a_ans)

q1b = 'What is proclaimed to be the "highest-grossing" film?'
q1b_ans = question_answerer(question=q1b, context=c1)
print('q1b', q1b_ans)

q1a {'score': 9.966933056659855e-09, 'start': 37, 'end': 54, 'answer': 'over $2.9 billion'}
q1b {'score': 7.906986732031385e-10, 'start': 0, 'end': 46, 'answer': 'With a worldwide box-office gross of over $2.9'}


In [17]:
q2 = 'How is Sandy feeling today?'
c2 = 'Jim said he feels absolutely awful.'
q2_ans = question_answerer(question=q2, context=c2)
print('q2', q2_ans)

q2 {'score': 0.0008811914012767375, 'start': 18, 'end': 34, 'answer': 'absolutely awful'}


In [18]:
# information from https://planetradio.co.uk/hits-radio/entertainment/music/taylor-swift-awards/
c3 = 'As one of the busiest women in music, it\'ll come as no surprise that Taylor has won pretty much every award there is to win in the biz - being the proud owner of no less than 12 GRAMMY Awards.'

q3 = 'How many Grammys has Taylor Swift won?'
q3_ans = question_answerer(question=q3, context=c3)
print('q3', q3_ans)

q3 {'score': 1.4922909485903801e-07, 'start': 162, 'end': 191, 'answer': 'no less than 12 GRAMMY Awards'}


In [19]:
# information from https://www.1057thepoint.com/music-news/hozier-announces-new-album-unreal-unearth/
c4 = 'Hozier has announced a new album called Unreal Unearth. The third full-length effort from the “Take Me to Church” artist — and his first since 2019\'s Wasteland, Baby! — arrives August 18.'

q4 = 'What is Hozier\'s new album?'
q4_ans = question_answerer(question=q4, context=c4)
print('q4', q4_ans)

q4 {'score': 3.425412273827533e-07, 'start': 40, 'end': 54, 'answer': 'Unreal Unearth'}


In [20]:
import wandb

# use to log in to wandb if needed
# API_KEY = # wandb api key
# wandb.login(key=API_KEY)

wandb.init(
    project='SQuAD2.0 with Fine-Tuned DistilBERT',
    notes='Solving Standford\'s SQuAD 2.0 Q&A dataset with DistilBERT transfer learning.',
)

wandb.config = {
    'epochs': EPOCHS, 
    'learning_rate': LR, 
    'batch_size': BATCH_SIZE,
    'weight_decay': WEIGHT_DECAY,
}

table = wandb.Table(columns=['Content', 'Question', 'Model Answer'], 
            data=[[c1, q1a, q1a_ans], [c1, q1b, q1b_ans], [c2, q2, q2_ans], [c3, q3, q3_ans], [c4, q4, q4_ans]])

wandb.log({ 'QA Inference': table })

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arth-shukla. Use `wandb login --relogin` to force relogin
